# graph.net._utils.node_lookup_by_address

> TODO fill in description

In [ ]:
#| default_exp graph._utils.node_lookup_by_address

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import ABC, abstractmethod
import typing
from types import MappingProxyType
from typing import Type, Optional, Callable, Any, Union, Tuple, Coroutine, List, Dict, NewType
from enum import Enum
import inspect
import uuid
from datetime import datetime, timezone
from inspect import signature
from fastcore.basics import patch_to

import fbdev
from fbdev.comp.port import PortType
from fbdev.graph.net import Address, Node, Net, NodePort, Edge

In [ ]:
#|export
__delimiters = {
    Node._address_delimiter : Node,
    Edge._address_delimiter : Edge,
    NodePort._address_delimiter : NodePort,
}
def _get_next_address_part(address:Address) -> Tuple[str, Address, Type]:
    delimiter_idx = next((i for i, c in enumerate(address) if c in __delimiters.keys()), -1)
    if delimiter_idx == -1: return address, '', None
    id, delimiter, rest_of_address = address[:delimiter_idx], address[delimiter_idx], address[delimiter_idx+1:]
    return id, rest_of_address, __delimiters[delimiter]

def _get_node_child_by_address(node:Node|Net, address:Address) -> Node|Edge|NodePort:
    id, rest_of_address, child_type = _get_next_address_part(address)
    child_id, rest_of_address_after_child, _ = _get_next_address_part(rest_of_address)
    
    if id == '': id = node.id
    if id != node.id: raise ValueError(f"First part of '{address}' does not match net id '{node.id}'.")
    
    if not child_id: return node
    else:
        if child_type == Node:
            if type(node) == Net:
                return node.nodes[child_id].get_child_by_address(rest_of_address_after_child)
            else:
                raise ValueError(f"Poorly formatted address '{address}'. Non-net cannot have child nodes.")
        elif child_type == Edge:
            if not rest_of_address_after_child: raise ValueError(f"Poorly formatted address '{address}'")
            return node.edges[child_id]
        elif child_type == NodePort:
            if not rest_of_address_after_child: raise ValueError(f"Poorly formatted address '{address}'")
            port_type_label, port_name = child_id.split('.')
            port_type = PortType.get(port_type_label)
            return node.ports[(port_type, port_name)]
        else:
            raise ValueError(f"Poorly formatted address '{address}'")

In [ ]:
id, rest_of_address, child_type = _get_next_address_part('NET.child|edge0')
assert id == 'NET'
assert rest_of_address == 'child|edge0'
assert child_type == Node